# Google Collab Environment Setup
## Change Runtime
Please run this Google Colab Jupyter Notebook with the following settings under "Runtime" -> "Change runtime type".

Runtime type: Python 3

Hardware accelerator: T4 GPU

## Setting up Python 3.8
This project uses Python 3.8 whilst Google Colab currently uses Python 3.10. Run the following commands to downgrade to Python 3.8.

Note: This process takes quite some time so you can leave it running in the background.

In [ ]:
# Downgrade Python by reinstalling pip and distutils
!apt-get install python3.8 python3-pip python3.8-distutils
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 1

# Check the result
!python --version

## Installing required dependencies
This notebook uses several libraries and dependencies. If you are using Google Collab, run the below commands to install the required libraries and dependencies before continuing:

In [ ]:
# Create button to upload requirement.txt
from google.colab import files
uploaded = files.upload()

In [ ]:
# install dependencies using current Python version (Python 3.8)
!python -m pip install -r requirements.txt